In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta, date
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from mapping_plot_attributes import theme_style
from mapping_tickers import *
from utils import *
from download_data import DownloadData

In [2]:
tickers = list(magnificent_7_tickers.keys())
end_date = datetime.today()
hist_years, hist_months, hist_days = 1, 0, 0
start_date = datetime(end_date.year - hist_years, end_date.month - hist_months, end_date.day - hist_days)
tk_market = '^GSPC'

hist_data = DownloadData(end_date, start_date, tickers, tk_market)

downloaded_data = hist_data.download_yh_data(start_date, end_date, tickers, tk_market)
df_adj_close = downloaded_data['Adj Close']
df_close = downloaded_data['Close']
dict_ohlc = downloaded_data['OHLC']

tk = 'AAPL'
df_ohlc = dict_ohlc[tk]
# display(df_adj_close)
# display(df_close)
# display(df_ohlc)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

The portfolio data will be truncated to end at the latest available date of 2024-09-18.


In [3]:
def moving_average(
    df_tk,
    ma_type,
    window,
    min_periods = 1
):
    """
    df_tk:      a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    ma_type:    simple ('sma'), exponential ('ema'), double exponential ('dema'), triple exponential ('tema')
    window:     length in days
    Returns ma
    """

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    if ma_type in ['ema', 'dema', 'tema']:
        ma = df_tk.ewm(span = window).mean()
        if ma_type in ['dema', 'tema']:
            ma = ma.ewm(span = window).mean()
            if ma_type == 'tema':
                ma = ma.ewm(span = window).mean()
    
    else:  # 'sma' or anything else
        ma = df_tk.rolling(window = window, min_periods = min_periods).mean()
    
    return ma

In [4]:
def plot_hist_plotly(
    df_price,
    tk,
    n_ticks_max = 48,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark',
    price_type = 'adjusted close'
):
    """
    price_type: one of ['adjusted close', 'close', 'open', 'high', 'low']

    """

    if isinstance(df_price, pd.Series):
        df_tk = df_price.copy()
    elif isinstance(df_price, pd.DataFrame):
        df_tk = df_price[tk]
    else:
        print('Incorrect format of input data')
        exit

    style = theme_style[theme]

    min_y = min(df_tk)
    max_y = max(df_tk)
    y_min, y_max = set_axis_limits(min_y, max_y)

    fig = make_subplots(rows = 1, cols = 1)

    fig.add_trace(
        go.Scatter(
            x = df_tk.index.astype(str),
            y = df_tk,
            line = dict(color = style['basecolor']),
            showlegend = True,
            name = price_type.title()
        )
    )
    # Add plot border
    fig.add_shape(
        type = 'rect',
        xref = 'paper',
        yref = 'paper',
        x0 = 0,
        x1 = 1,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    # Update layout and axes
    fig.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        yaxis_title = f'{price_type.title()}',
        title = dict(
            text = f'{tk} {price_type.title()}',
            font_size = title_font_size,
            y = 0.95,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],        
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        gridcolor = style['y_gridcolor'],
        ticks = 'outside',
        ticklen = 8
    )

    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max
    }

    return fig_data

In [5]:
fig_hist_data = plot_hist_plotly(df_adj_close, tk)
fig_hist = fig_hist_data['fig']
fig_hist.show()

In [6]:
def add_overlay(
    fig_data,
    df,
    name,
    color_idx,
    showlegend = True,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data: a dictionary of the underlying figure data

    y_min_fig: y_min on the existing fig
    y_max_fig: y_max on the existing fig
    color_idx: an integer (0, ...) indicating the color from those available in theme_style
    showlegend: whether or not to show line in legend (e.g. we only need one Bollinger band in legend)

    Returns the updated fig_data dictionary
    """

    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]

    fig = fig_data['fig']
    y_min_fig = fig_data['y_min']
    y_max_fig = fig_data['y_max']

    min_y = min(df)
    max_y = max(df)
    y_min, y_max = set_axis_limits(min_y, max_y)

    new_y_min, new_y_max = min(y_min, y_min_fig), max(y_max, y_max_fig)

    if color_idx >= len(overlay_colors):
        # Take the last overlay color from the available list
        color_idx = -1

    fig.add_trace(
        go.Scatter(
            x = df.index.astype(str),
            y = df,
            line = dict(color = overlay_colors[color_idx]),
            name = name,
            showlegend = showlegend
        )
    )

    fig.update_yaxes(
        range = (new_y_min, new_y_max),
        gridcolor = style['y_gridcolor'],
        ticks = 'outside',
        ticklen = 8
    )

    fig_data = {
        'fig': fig,
        'y_min': new_y_min,
        'y_max': new_y_max
    }

    return fig_data

In [7]:
overlays = []
overlay_1 = {
    'window': 12,
    'type': 'ema',
    'color_idx': 1
}
df_1 = moving_average(
    df_adj_close[tk],
    ma_type = overlay_1['type'],
    window = overlay_1['window']
)
name_1 = f"{overlay_1['window']}-Day {overlay_1['type'].upper()}"
overlay_1.update({'name': name_1})
overlay_1.update({'data': df_1})
overlays.append(overlay_1)

overlay_2 = {
    'window': 26,
    'type': 'ema',
    'color_idx': 2
}
df_2 = moving_average(
    df_adj_close[tk],
    ma_type = overlay_2['type'],
    window = overlay_2['window']
)
name_2 = f"{overlay_2['window']}-Day {overlay_2['type'].upper()}"
overlay_2.update({'name': name_2})
overlay_2.update({'data': df_2})
overlays.append(overlay_2)

fig_data = plot_hist_plotly(df_adj_close, tk)
for overlay in overlays:
    fig_data = add_overlay(fig_data, overlay['data'], overlay['name'], overlay['color_idx'])

fig = fig_data['fig']
fig.show()

Test Bollinger Band Overlays

Candlestick

In [8]:
def plot_candlestick_plotly(
    df_ohlc,
    tk,
    candle_type = 'hollow',
    n_ticks_max = 48,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark'
):
    """
    candle_type: 'hollow' or 'traditional'
        
    """
    
    style = theme_style[theme]
    red_color = style['red_color']
    green_color = style['green_color']

    df = df_ohlc.copy()

    min_y = min(df['Low'])
    max_y = max(df['High'])
    y_min, y_max = set_axis_limits(min_y, max_y)

    df['Date'] = df.index.astype(str)
    x_min = df['Date'].min()
    x_max = df['Date'].max()

    fig = make_subplots(rows = 1, cols = 1)

    if candle_type == 'traditional':
        
        title = f'{tk} Prices - Traditional Candles'

        shown_green = False
        shown_red = False

        for idx, row in df.iterrows():

            if row['Close'] >= row['Open']:
                color_dict = dict(
                    fillcolor = 'rgba(0, 255, 0, 0.3)',
                    line = dict(color = green_color)
                )
                name = 'Close > Open'
                current_candle = 'green'
            else:
                color_dict = dict(
                    fillcolor = 'rgba(255, 0, 0, 0.6)',
                    line = dict(color = red_color)
                )
                name = 'Open > Close'
                current_candle = 'red'

            # Make sure each candle type appears only once in the legend
            if (not shown_green) & (current_candle == 'green'):
                showlegend = True
                shown_green = True
            elif (not shown_red) & (current_candle == 'red'):
                showlegend = True
                shown_red = True
            else:
                showlegend = False

            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend
                )
            )
        
    else:  # candle_type == 'hollow'
        
        title = f'{tk} Prices - Hollow Candles'
        
        df['previousClose'] = df['Close'].shift(1)
        
        # Define color based on close and previous close
        df['color'] = np.where(df['Close'] > df['previousClose'], green_color, red_color)
        
        # Set fill to transparent if close > open and the previously defined color otherwise
        df['fill'] = np.where(df['Close'] > df['Open'], 'rgba(255, 0, 0, 0)', df['color'])
        
        shown_red_fill = False
        shown_red_hollow = False
        shown_green_fill = False
        shown_green_hollow = False
        
        for idx, row in df.iterrows():
            
            if (row['color'] == green_color) & (row['fill'] == green_color):
                name = 'Open > Close > Prev Close'
                current_candle = 'green_fill'
            elif (row['color'] == green_color) & (row['fill'] == 'rgba(255, 0, 0, 0)'):
                name = 'Prev Close < Close > Open'
                current_candle = 'green_hollow'
            elif (row['color'] == red_color) & (row['fill'] == red_color):
                name = 'Open > Close < Prev Close'
                current_candle = 'red_fill'
            elif (row['color'] == red_color) & (row['fill'] == 'rgba(255, 0, 0, 0)'):
                name = 'Prev Close > Close > Open'
                current_candle = 'red_hollow'
            else:
                name = 'Hollow Candles'
            
            # Make sure each candle type appears only once in the legend
            if (not shown_green_fill) & (current_candle == 'green_fill'):
                showlegend = True
                shown_green_fill = True
            elif (not shown_green_hollow) & (current_candle == 'green_hollow'):
                showlegend = True
                shown_green_hollow = True
            elif (not shown_red_fill) & (current_candle == 'red_fill'):
                showlegend = True
                shown_red_fill = True
            elif (not shown_red_hollow) & (current_candle == 'red_hollow'):
                showlegend = True
                shown_red_hollow = True
            else:
                showlegend = False
        
            color_dict = dict(
                fillcolor = row['fill'],
                line=dict(color = row['color'])
            )
            
            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    name = name,
                    legendgroup = 'Hollow Candlesticks'
                )
            )
        
    # Add plot border
    fig.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' to avoid double lines at x_min and x_max
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    # Update layout and axes
    fig.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        yaxis_title = f'Price',
        title = dict(
            text = title,
            font_size = title_font_size,
            y = 0.95,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],        
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        gridcolor = style['y_gridcolor'],
        ticks = 'outside',
        ticklen = 8
    )

    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max
    }

    return fig_data

In [9]:
candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'hollow', theme = 'dark')
candle_fig = candle_data['fig']
candle_fig.show()

In [10]:
def bollinger_bands(
    prices,
    window = 20,
    n_std = 2.0,
    n_bands = 1
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    window:
        size of the rolling window in days, defaults to 20
    n_std:
        width of the upper and lower bands in standard deviations, defaults to 2.0
    n_bands:
        number of pairs of bands to be created, defaults to 1, max 3

    Returns a list of bollinger band dictionaries
    """

    eps = 1e-6

    n_bands = min(3, n_bands)

    df_sma = prices.rolling(window = window, min_periods = 1).mean()
    df_std = prices.rolling(window = window, min_periods = 1).std(ddof=0)
    
    bollinger_list = [{
        'data': df_sma,
        'name': f'SMA {window}',
        'idx_offset': 0,
        'showlegend': True
    }]

    for i in range(n_bands + 1)[1:]:
        
        band_width = i * n_std
        k = 0 if abs(float(int(band_width)) - band_width) < eps else 1
        # k = 0: ma_offset is an integer within the accuray of eps

        upper_band = df_sma + band_width * df_std
        upper_name = f'({window}, {band_width:.{k}f}) Bollinger Bands'
        bollinger_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i,
            'showlegend': True
        })

        lower_band = df_sma - band_width * df_std        
        lower_name = f'({window}, {band_width:.{k}f}) Lower Band'
        bollinger_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i,
            'showlegend': False
        })

    return bollinger_list

In [11]:
def add_bollinger_overlays(
    fig_data,
    bollinger_list,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: df_close or df_adj_close, depending on the underlying figure in fig_data

    """
    n_boll = int((len(bollinger_list) + 1) / 2)
    print(n_boll)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_boll]
    
    bollinger_overlays = []
    
    for boll in bollinger_list:
        
        bollinger_overlays.append({
            'data': boll['data'],
            'name': boll['name'],
            'color_idx': overlay_color_idx[abs(boll['idx_offset'])],
            'showlegend': boll['showlegend']
        })

    color_map = {}

    for overlay in bollinger_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            theme = theme,
            color_theme = color_theme
        )
        color_map.update({overlay['name']: overlay['color_idx']})

    fig_data.update({'color_map': color_map})

    return fig_data

In [20]:
theme = 'light'
theme = 'dark'
color_theme = 'gold'
color_theme = 'gold'

bollinger_list = bollinger_bands(df_adj_close[tk], 20, 2.1, 3)
fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_boll_new = add_bollinger_overlays(fig_hist_data, bollinger_list, theme = theme, color_theme = color_theme)

fig_boll_new['fig'].show()

4


In [13]:
def ma_envelopes(
    prices,
    ma_type = None,
    window = 20,
    prc_offset = 5,
    n_bands = 3
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    ma_type:
        one of 'sma', 'ema', dema', tema'
    window:
        size of the rolling window in days
    prc_offset: 
        vertical offset from base moving average in percentage points (-99% to 99%)
    n_bands:
        number of pairs of envelopes to be created, defaults to 3 (max)

    Returns a list of ma envelope dictionaries
    """

    eps = 1e-6
    
    if ma_type is None:
        ma_type = 'sma'

    n_bands = min(3, n_bands)
    if abs(prc_offset) > 99:
        prc_offset = math.sign(prc_offset) * 99

    base_ma = moving_average(prices, ma_type, window)

    base_name = f'{ma_type.upper()} {window}'
    
    ma_envelope_list = [{
        'data': base_ma,
        'name': base_name,
        'idx_offset': 0,
        'showlegend': True
    }]

    for i in range(n_bands + 1)[1:]:
        
        ma_offset = i * prc_offset
        k = 0 if abs(float(int(ma_offset)) - ma_offset) < eps else 1
        # k = 0: ma_offset is an integer within the accuray of eps

        upper_band = base_ma * (1 + ma_offset / 100)
        upper_name = f'({window}, {ma_offset:.{k}f}%) Envelopes'
        ma_envelope_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i,
            'showlegend': True
        })

        lower_band = base_ma * (1 - ma_offset / 100)
        lower_name = f'({window}, {ma_offset:.{k}f}% Lower Envelope'
        ma_envelope_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i,
            'showlegend': False
        })

    return ma_envelope_list

In [14]:
def add_ma_envelope_overlays(
    fig_data,
    ma_envelope_list,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: df_close or df_adj_close, depending on the underlying figure in fig_data

    """
    n_env = int((len(ma_envelope_list) + 1) / 2)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_env]
    
    ma_envelope_overlays = []
    
    for env in ma_envelope_list:
        
        ma_envelope_overlays.append({
            'data': env['data'],
            'name': env['name'],
            'color_idx': overlay_color_idx[abs(env['idx_offset'])],
            'showlegend': env['showlegend']
        })

    color_map = {}

    for overlay in ma_envelope_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            theme = theme,
            color_theme = color_theme
        )
        color_map.update({overlay['name']: overlay['color_idx']})

    fig_data.update({'color_map': color_map})

    return fig_data

In [25]:
theme = 'light'
theme = 'dark'
color_theme = 'gold'
color_theme = 'gold'

# ma_type = 'sma'
window = 20
prc_offset = 2.5
n_bands = 3

# ma_envelope_list = ma_envelopes(df_adj_close[tk], ma_type, window, prc_offset = prc_offset, n_bands = n_bands)
ma_envelope_list = ma_envelopes(df_adj_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_ma_env = add_ma_envelope_overlays(fig_hist_data, ma_envelope_list, theme = theme, color_theme = color_theme)

fig_ma_env['fig'].show()

bollinger_list = bollinger_bands(df_adj_close[tk], 20, 2, 2)
fig_boll_new = add_bollinger_overlays(fig_ma_env, bollinger_list, theme = theme, color_theme = 'turquoise')

fig_boll_new['fig'].show()

3


In [22]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import theme_style

analyze_prices = AnalyzePrices(end_date, start_date, [tk])
theme = 'dark'
color_theme = 'turquoise'
style= theme_style[theme]

# candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
candle_data = analyze_prices.plot_candlestick_plotly(df_ohlc, tk, candle_type = 'hollow', theme = 'dark')
candle_fig = candle_data['fig']

bollinger_list = bollinger_bands(df_close[tk], 20, 2, 2)
candle_data = analyze_prices.add_bollinger_overlays(candle_data, bollinger_list, theme = theme, color_theme = color_theme)

fig_boll = candle_data['fig']

fig_boll.show()

Moving Average Overlays

In [20]:
def add_ma_overlays(
    fig_data,
    df_price,
    ma_list,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: 
        df_close or df_adj_close, depending on the underlying figure
    ma_list: 
        list of ma overlay dictionaries, containing
         - ma_idx ma index (1, 2,...)
         - ma_type: 'sma' (default), 'ema', 'dema', or 'tema'
         - ma_window, in days
         - showlegend: include in plot legend or not
    """
    
    n_ma = len(ma_list)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_ma]

    ma_overlays = []

    for i, ma in enumerate(ma_list):
        
        ma_type = ma['ma_type']
        ma_window = ma['ma_window']

        ma_data = moving_average(
            df_price,
            ma_type,
            ma_window
        )
        ma_name = f'{ma_type.upper()} {ma_window}'
        ma_color_idx = overlay_color_idx[i]
        ma_showlegend = ma['showlegend']

        ma_overlays.append({
            'data': ma_data,
            'name': ma_name,
            'color_idx': ma_color_idx,
            'showlegend': ma_showlegend
        })

    color_map = {}

    for overlay in ma_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            theme = theme,
            color_theme = color_theme
        )        
        color_map.update({overlay['name']: overlay['color_idx']})

    fig_data.update({'color_map': color_map})

    return fig_data

In [18]:
ma_list = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'sma',
        'ma_window': 20,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'sma',
        'ma_window': 30,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'sma',
        'ma_window': 40,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'sma',
        'ma_window': 50,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'sma',
        'ma_window': 60,
        'showlegend': True
    }
]


In [21]:
# theme = 'light'
theme = 'dark'
color_theme = 'tableau'
from mapping_plot_attributes import theme_style
from analyze_prices import AnalyzePrices
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = theme)
# candle_data = analyze_prices.plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = theme)
candle_data = add_ma_overlays(candle_data, df_close[tk], ma_list, theme = theme, color_theme = color_theme)
candle_fig = candle_data['fig']
candle_fig.show()

In [22]:
from mapping_plot_attributes import theme_style
from analyze_prices import AnalyzePrices
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

theme = 'light'
# theme = 'dark'
color_theme = 'tableau'

fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_hist = fig_hist_data['fig']

new_data = add_ma_overlays(fig_hist_data, df_adj_close[tk], ma_list, theme = theme, color_theme = color_theme)
new_data_fig = new_data['fig']
new_data_fig.show()

In [21]:
print(new_data_fig['data'])

(Scatter({
    'line': {'color': '#1f77b4'},
    'name': 'Adjusted Close',
    'showlegend': True,
    'x': array(['2023-09-18', '2023-09-19', '2023-09-20', ..., '2024-09-11',
                '2024-09-12', '2024-09-13'], dtype=object),
    'y': array([177.06394958, 178.15834045, 174.59658813, ..., 222.66000366,
                222.77000427, 222.5       ])
}), Scatter({
    'line': {'color': 'rgb(23, 190, 207)'},
    'name': '10-Day SMA',
    'showlegend': True,
    'x': array(['2023-09-18', '2023-09-19', '2023-09-20', ..., '2024-09-11',
                '2024-09-12', '2024-09-13'], dtype=object),
    'y': array([177.06394958, 177.61114502, 176.60629272, ..., 223.57800293,
                223.20600281, 222.47700348])
}), Scatter({
    'line': {'color': 'rgb(188, 189, 34)'},
    'name': '20-Day SMA',
    'showlegend': True,
    'x': array(['2023-09-18', '2023-09-19', '2023-09-20', ..., '2024-09-11',
                '2024-09-12', '2024-09-13'], dtype=object),
    'y': array([177.06394958, 

In [22]:
print(new_data['color_map'])

{'10-Day SMA': 5, '20-Day SMA': 4, '30-Day SMA': 3, '40-Day SMA': 2, '50-Day SMA': 1, '60-Day SMA': 0}


In [23]:
def update_color_theme(
    fig,
    color_map,
    theme,
    color_theme,
    invert = False
):
    """
    fig = fig_data['fig']
    color_map = fig_data['color_map']: an overlay color map dictionary
    theme: existing theme ('dark' or light')
    color_theme: new color theme to apply to overlays in fig
    invert: invert the palette from lightest-darkest to darkest-lightest or vice versa?

    Returns updated fig
    """
    
    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]

    for name, color_idx in color_map.items():
        
        if invert:
            color_idx = len(color_map) - color_idx - 1

        fig.update_traces(
            line_color = overlay_colors[color_idx],
            selector = dict(name = name)
        )
    
    return fig

In [24]:
# https://plotly.com/python/creating-and-updating-figures/

from mapping_plot_attributes import theme_style

theme = 'light'
# theme = 'dark'
style= theme_style[theme]
new_color_theme = 'tableau'

new_data_fig = update_color_theme(new_data_fig, new_data['color_map'], theme, new_color_theme, invert = True)
# new_data_fig = update_color_theme(new_data_fig, new_data['color_map'], theme, new_color_theme)

new_data_fig.show()